In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('../data/final/pat_adm_lab_vital.csv')
df.head()

,patientunitstayid,aki_label,aki_offset,organsys_Cardiovascular,organsys_Gastrointestinal,organsys_Genitourinary,organsys_Hematology,organsys_Metabolic/Endocrine,organsys_Musculoskeletal/Skin,organsys_Neurologic,...,delta_result_sodium,min_sao2,max_sao2,mean_sao2,min_heartrate,max_heartrate,mean_heartrate,min_respiration,max_respiration,mean_respiration
0,141284,0,NaN,0,0,0,1,0,0,0,...,0.0,92.0,100.0,96.073529,70.0,106.0,84.752613,10.0,36.0,21.254355
1,141288,0,NaN,1,0,0,0,0,0,0,...,0.0,90.0,100.0,95.219512,80.0,106.0,91.857143,11.0,46.0,21.972125
2,141360,0,NaN,0,0,0,0,0,0,1,...,0.0,96.0,100.0,98.602230,76.0,120.0,89.808118,11.0,36.0,17.656827
3,141362,0,NaN,0,1,0,0,0,0,0,...,2.0,80.0,99.0,93.338129,92.0,120.0,103.592982,11.0,42.0,24.518182
4,141415,0,NaN,1,0,0,0,0,0,0,...,2.0,97.0,100.0,98.353488,61.0,99.0,72.034722,8.0,57.0,15.880000
